In [2]:
%%time
# Underlying details extracted from nse
# Program takes 5 mins

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4
import json
import datetime as datetime
import sys

#....        Initializations       ....
#......................................

load_from_disk = True   # For underlying symbol list


#......  Error catch in list comprehension  ...

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        pass

#.....    get / make the symbols list  .....

if load_from_disk:
    try:
        symbols = list(pd.read_csv('./zdata/nse_eq_symbols.csv'))
    except FileNotFoundError as e:
        print('csv file not found')
        sys.exit(0)

#.....    get the underlying symbols   .....

def get_underlying(symbol):
    
    '''Gets the underlying data
    Arg: (symbol as string)
    Returns: df as dataframe'''

    # URLs
    url_base1 = "https://www.nseindia.com/live_market/dynaContent/live_watch/"
    url = url_base1 + "get_quote/GetQuote.jsp?symbol="+symbol

    page = requests.get(url).text

    # soup out the json dict
    bs_nse = bs4(page, 'html.parser')
    json_nse = json.loads(bs_nse.find(id='responseDiv').text.strip())

    df = pd.DataFrame.from_dict(json_nse['data'][0], orient='index').T
    
    return df

nse_underlyings = [catch(lambda: get_underlying(symbol)) for symbol in symbols]

# indexed by symbols
df_nse_und = pd.concat(nse_underlyings).set_index('symbol')

# clean up the commas, dashes and empty strs
df_nse_und1 = df_nse_und.replace(',|-' , '', regex=True)
df_nse_und1 = df_nse_und1.replace('', np.nan)

# convert date columns to datetime.date format
filtin = ['Date', 'dt']
filtout = ['isExDateFlag']
date_columns_mask = df_nse_und1.columns.str.contains('|'.join(filtin)) | (df_nse_und1.columns.str.contains('|'.join(filtout)))
dt_col_list = df_nse_und1.loc[:, date_columns_mask].columns.tolist()
dt_col_list = [item for item in dt_col_list if item not in filtout]  #filterout

df_nse_und1.loc[:, dt_col_list] = df_nse_und1.loc [:, dt_col_list].apply( \
                                  pd.to_datetime, errors= 'coerce').applymap(pd.Timestamp.date)

dict_df = df_nse_und1.to_dict(orient='index')  # dictionary based on index of symbols

# store data for next program
df_nse_und1.to_pickle('./zdata/underlying_df.pkl')

Wall time: 2min 19s


In [96]:
# ignored because dataframe pickle occupies lesser space!
# import pickle
# with open('./zdata/underlying_dict.pkl', 'wb') as handle:
#     pickle.dump(dict_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
df_nse_und1

,pricebandupper,applicableMargin,bcEndDate,totalSellQuantity,adhocMargin,companyName,marketType,exDate,bcStartDate,css_status_desc,...,buyQuantity3,buyQuantity2,buyQuantity1,series,faceValue,buyQuantity5,closePrice,open,isinCode,lastPrice
symbol,,,,,,,,,,,,,,,,,,,,,
ACC,1504.35,14.16,2018-05-25,326,NaN,ACC Limited,N,2018-05-17,2018-05-21,Listed,...,NaN,NaN,NaN,EQ,10.00,NaN,1349.65,1370.00,INE012A01025,1349.95
INDIANB,240.15,18.71,2018-06-28,1595,NaN,Indian Bank,N,2018-06-21,2018-06-23,Listed,...,NaN,NaN,NaN,EQ,10.00,NaN,222.30,218.40,INE562A01011,221.00
DISHTV,40.60,29.44,2018-09-27,18073,NaN,Dish TV India Limited,N,2018-09-21,2018-09-25,Listed,...,NaN,NaN,NaN,EQ,1.00,NaN,40.90,36.95,INE836F01026,40.80
SHREECEM,14883.15,13.53,2018-07-30,NaN,NaN,SHREE CEMENT LIMITED,N,2018-07-20,2018-07-24,Listed,...,NaN,NaN,215,EQ,10.00,NaN,13568.85,13530.15,INE070A01015,13760.00
HCC,13.00,25.16,2018-07-12,NaN,NaN,Hindustan Construction Company Limited,N,2018-07-04,2018-07-06,Listed,...,NaN,NaN,638,EQ,1.00,NaN,11.85,11.90,INE549A01026,11.90
M&M,807.10,13.81,2018-08-07,2474,NaN,Mahindra & Mahindra Limited,N,2018-07-12,2018-07-14,Listed,...,NaN,NaN,NaN,EQ,5.00,NaN,728.20,731.00,INE101A01026,727.00
CAPF,508.30,15.50,2018-06-28,83,NaN,Capital First Limited,N,2018-06-26,2018-06-28,Listed,...,NaN,NaN,NaN,EQ,10.00,NaN,450.45,462.10,INE688I01017,448.75
MCDOWELL-N,589.05,14.25,2018-09-07,2502,NaN,United Spirits Limited,N,2018-08-30,2018-09-01,Listed,...,NaN,NaN,NaN,EQ,2.00,NaN,518.05,537.00,INE854D01024,516.50
MUTHOOTFIN,427.80,16.70,2018-09-29,NaN,NaN,Muthoot Finance Limited,N,2018-09-19,2018-09-23,Listed,...,NaN,NaN,931,EQ,10.00,NaN,398.75,388.95,INE414G01012,393.40
